# Triangle Factorization of Factorials Using `milp()`

In [1]:
import numpy as np
import scipy
import sympy
from scipy.optimize import LinearConstraint, milp

def generateTriangleNumbers(upToN):
    result = 1
    i = 2
    while result < upToN:
        yield result
        result += i
        i += 1

def generateTriangleDivisors(ofN, triangleNumbersUpToN = None):
    for candidate in generateTriangleNumbers(
        ofN // 2 if triangleNumbersUpToN is None else triangleNumbersUpToN
    ):
        if ofN % candidate == 0:
            yield candidate
            
def factorial(n):
    p = 1
    for i in range(1, n + 1):
        p *= i
    return p

fac1993 = factorial(1993)

In [2]:
#divs = list(generateTriangleDivisors(fac1993, triangleNumbersUpToN=10**14))
#print(len(divs))
#with open('1993_divisors_to_10_to_14.txt', 'w') as fout:
#    fout.write(str(divs))

with open('1993_divisors_to_10_to_14.txt', 'r') as fin:
    danger = fin.read()
    divs = eval(danger)
    print(len(divs))
    divs = divs[:19680] # For memory constraints.


946382


In [3]:
fac1993factor = sympy.factorint(fac1993)
fac1993primesVector = list(value for value in fac1993factor)
def toPrimeVector(n):
    factor = sympy.factorint(n)
    return [factor[p] if p in factor else 0 for p in fac1993factor]

In [4]:
As = []
ubs = []
lbs = []

for i in range(len(divs)):
    ThisA = [0] * len(divs)
    ThisA[i] = 1
    As.append(ThisA)
    ubs.append(1)
    lbs.append(0)

for p in fac1993factor:
    ThisA = [0] * len(divs)
    for i in range(len(divs)):
        factor = sympy.factorint(divs[i])
        ThisA[i] += factor[p] if p in factor else 0
    As.append(ThisA) 
    ubs.append(fac1993factor[p])
    lbs.append(fac1993factor[p])

c = np.array([1] * len(divs))
A = np.array(As)
b_u = np.array(ubs)
b_l = np.array(lbs)

del As
del ubs
del lbs

In [5]:
constraints = LinearConstraint(A, b_l, b_u)

integrality = np.array([1] * len(divs))

res = milp(c=c, constraints=constraints, integrality=integrality)

res.x

array([0., 0., 0., ..., 1., 1., 1.])

In [6]:
ps = []
p = 1
for i, x in enumerate(res.x):
    if x > 0.1:
        p *= divs[i]
        ps.append(divs[i])
print(ps)

[4522528, 21526641, 99313371, 209091025, 301977600, 354432000, 368167680, 382219776, 396506880, 411027456, 411056128, 425838336, 425867520, 431460000, 433371520, 440941056, 448546176, 448576128, 456276736, 464012416, 464042880, 467919936, 468930000, 471843840, 471874560, 475814976, 479740800, 483713856, 487734528, 493749600, 495731328, 501795360, 503856640, 505858528, 505890336, 506908720, 507896256, 514066080, 516120256, 522275040, 524345536, 524377920, 524863800, 526452576, 527491440, 528531328, 534759456, 534792160, 535839216, 536854528, 536887296, 537903600, 538067610, 541056960, 541089856, 541616328, 542110128, 542143056, 545308800, 547424416, 549544128, 551634720, 553230216, 553795840, 555927840, 556962000, 559100080, 560204128, 562314880, 562348416, 562885128, 563153580, 565000920, 566615616, 567693360, 568232616, 569851920, 570966528, 572048400, 573097440, 574181328, 575266240, 576929496, 577473120, 578017000, 579616128, 579650176, 580706160, 580740240, 581422050, 581797216, 58

In [8]:
print(p == fac1993)

True
